In [35]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import pathlib
import tree as miptree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from model.threshold_guess import compute_thresholds, cut
from model.gosdt import GOSDT
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Data Preprocessing

### Default Data

In [36]:
default_data = pd.read_csv('/Users/andrew/Documents/MBS/Practicum/Data/Default/default_data.csv') 

In [37]:
# Setting column limit for easy viewing to bypass limit

pd.set_option('display.max_columns', 500)
default_data.describe()

,UNIQUEID,DISBURSED_AMOUNT,ASSET_COST,LTV,BRANCH_ID,SUPPLIER_ID,MANUFACTURER_ID,CURRENT_PINCODE_ID,STATE_ID,EMPLOYEE_CODE_ID,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,VOTERID_FLAG,DRIVING_FLAG,PASSPORT_FLAG,PERFORM_CNS_SCORE,PRI_NO_OF_ACCTS,PRI_ACTIVE_ACCTS,PRI_OVERDUE_ACCTS,PRI_CURRENT_BALANCE,PRI_SANCTIONED_AMOUNT,PRI_DISBURSED_AMOUNT,SEC_NO_OF_ACCTS,SEC_ACTIVE_ACCTS,SEC_OVERDUE_ACCTS,SEC_CURRENT_BALANCE,SEC_SANCTIONED_AMOUNT,SEC_DISBURSED_AMOUNT,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,NEW_ACCTS_IN_LAST_SIX_MONTHS,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,NO_OF_INQUIRIES,LOAN_DEFAULT
count,233154.000000,233154.000000,2.331540e+05,233154.000000,233154.000000,233154.000000,233154.000000,233154.000000,233154.000000,233154.000000,233154.0,233154.00000,233154.000000,233154.000000,233154.000000,233154.000000,233154.000000,233154.000000,233154.000000,233154.000000,2.331540e+05,2.331540e+05,2.331540e+05,233154.000000,233154.000000,233154.000000,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,233154.000000,233154.000000,233154.000000,233154.000000
mean,535917.573376,54356.993528,7.586507e+04,74.746530,72.936094,19638.635035,69.028054,3396.880247,7.262243,1549.477148,1.0,0.84032,0.075577,0.144943,0.023242,0.002127,289.462994,2.440636,1.039896,0.156549,1.659001e+05,2.185039e+05,2.180659e+05,0.059081,0.027703,0.007244,5.427793e+03,7.295923e+03,7.179998e+03,1.310548e+04,3.232684e+02,0.381833,0.097481,0.206615,0.217071
std,68315.693711,12971.314171,1.894478e+04,11.456636,69.834995,3491.949566,22.141304,2238.147502,4.482230,975.261278,0.0,0.36631,0.264320,0.352044,0.150672,0.046074,338.374779,5.217233,1.941496,0.548787,9.422736e+05,2.374794e+06,2.377744e+06,0.626795,0.316057,0.111079,1.702370e+05,1.831560e+05,1.825925e+05,1.513679e+05,1.555369e+04,0.955107,0.384439,0.706498,0.412252
min,417428.000000,13320.000000,3.700000e+04,10.030000,1.000000,10524.000000,45.000000,1.000000,1.000000,1.000000,1.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.678296e+06,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,-5.746470e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,476786.250000,47145.000000,6.571700e+04,68.880000,14.000000,16535.000000,48.000000,1511.000000,4.000000,713.000000,1.0,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
50%,535978.500000,53803.000000,7.094600e+04,76.800000,61.000000,20333.000000,86.000000,2970.000000,6.000000,1451.000000,1.0,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
75%,595039.750000,60413.000000,7.920175e+04,83.670000,130.000000,23000.000000,86.000000,5677.000000,10.000000,2362.000000,1.0,1.00000,0.000000,0.000000,0.000000,0.000000,678.000000,3.000000,1.000000,0.000000,3.500650e+04,6.250000e+04,6.080000e+04,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.999000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000
max,671084.000000,990572.000000,1.628992e+06,95.000000,261.000000,24803.000000,156.000000,7345.000000,22.000000,3795.000000,1.0,1.00000,1.000000,1.000000,1.000000,1.000000,890.000000,453.000000,144.000000,25.000000,9.652492e+07,1.000000e+09,1.000000e+09,52.000000,36.000000,8.000000,3.603285e+07,3.000000e+07,3.000000e+07,2.564281e+07,4.170901e+06,35.000000,20.000000,36.000000,1.000000


In [38]:
default_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233154 entries, 0 to 233153
Data columns (total 41 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   UNIQUEID                             233154 non-null  int64  
 1   DISBURSED_AMOUNT                     233154 non-null  int64  
 2   ASSET_COST                           233154 non-null  int64  
 3   LTV                                  233154 non-null  float64
 4   BRANCH_ID                            233154 non-null  int64  
 5   SUPPLIER_ID                          233154 non-null  int64  
 6   MANUFACTURER_ID                      233154 non-null  int64  
 7   CURRENT_PINCODE_ID                   233154 non-null  int64  
 8   DATE_OF_BIRTH                        233154 non-null  object 
 9   EMPLOYMENT_TYPE                      225493 non-null  object 
 10  DISBURSAL_DATE                       233154 non-null  object 
 11  STATE_ID     

In [39]:
# Test

In [40]:
default_data.nunique()

UNIQUEID                               233154
DISBURSED_AMOUNT                        24565
ASSET_COST                              46252
LTV                                      6579
BRANCH_ID                                  82
SUPPLIER_ID                              2953
MANUFACTURER_ID                            11
CURRENT_PINCODE_ID                       6698
DATE_OF_BIRTH                           15433
EMPLOYMENT_TYPE                             2
DISBURSAL_DATE                             84
STATE_ID                                   22
EMPLOYEE_CODE_ID                         3270
MOBILENO_AVL_FLAG                           1
AADHAR_FLAG                                 2
PAN_FLAG                                    2
VOTERID_FLAG                                2
DRIVING_FLAG                                2
PASSPORT_FLAG                               2
PERFORM_CNS_SCORE                         573
PERFORM_CNS_SCORE_DESCRIPTION              20
PRI_NO_OF_ACCTS                   

In [41]:
fig = go.Figure(data=[go.Pie(labels=default_data.LOAN_DEFAULT, hole=.3)])
fig.add_annotation(text='LOAN_DEFAULT',
                   x=0.5,y=0.5,showarrow=False,font_size=14,opacity=0.7,font_family='monospace')
fig.update_traces(hoverinfo='label+percent+value',
                  marker=dict(colors=['darkorange','blue'], line=dict(color='#000000', width=2)))
fig.show()

In [42]:
default_data.head(10)

,UNIQUEID,DISBURSED_AMOUNT,ASSET_COST,LTV,BRANCH_ID,SUPPLIER_ID,MANUFACTURER_ID,CURRENT_PINCODE_ID,DATE_OF_BIRTH,EMPLOYMENT_TYPE,DISBURSAL_DATE,STATE_ID,EMPLOYEE_CODE_ID,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,VOTERID_FLAG,DRIVING_FLAG,PASSPORT_FLAG,PERFORM_CNS_SCORE,PERFORM_CNS_SCORE_DESCRIPTION,PRI_NO_OF_ACCTS,PRI_ACTIVE_ACCTS,PRI_OVERDUE_ACCTS,PRI_CURRENT_BALANCE,PRI_SANCTIONED_AMOUNT,PRI_DISBURSED_AMOUNT,SEC_NO_OF_ACCTS,SEC_ACTIVE_ACCTS,SEC_OVERDUE_ACCTS,SEC_CURRENT_BALANCE,SEC_SANCTIONED_AMOUNT,SEC_DISBURSED_AMOUNT,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,NEW_ACCTS_IN_LAST_SIX_MONTHS,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,AVERAGE_ACCT_AGE,CREDIT_HISTORY_LENGTH,NO_OF_INQUIRIES,LOAN_DEFAULT
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-1984,Salaried,03-08-2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-1985,Self employed,26-09-2018,6,1998,1,1,0,0,0,0,598,I-Medium Risk,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-1985,Self employed,01-08-2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-1993,Self employed,26-10-2018,6,1998,1,1,0,0,0,0,305,L-Very High Risk,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-1977,Self employed,26-09-2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1
5,518279,54513,61900,89.66,67,22807,45,1501,08-09-1990,Self employed,19-09-2018,6,1998,1,1,0,0,0,0,825,A-Very Low Risk,2,0,0,0,0,0,0,0,0,0,0,0,1347,0,0,0,1yrs 9mon,2yrs 0mon,0,0
6,529269,46349,61500,76.42,67,22807,45,1502,01-06-1988,Salaried,23-09-2018,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
7,510278,43894,61900,71.89,67,22807,45,1501,04-10-1989,Salaried,16-09-2018,6,1998,1,1,0,0,0,0,17,Not Scored: Not Enough Info available on the c...,1,1,0,72879,74500,74500,0,0,0,0,0,0,0,0,0,0,0yrs 2mon,0yrs 2mon,0,0
8,490213,53713,61973,89.56,67,22807,45,1497,15-11-1991,Self employed,05-09-2018,6,1998,1,1,0,0,0,0,718,D-Very Low Risk,1,1,0,-41,365384,365384,0,0,0,0,0,0,0,0,0,0,4yrs 8mon,4yrs 8mon,1,0
9,510980,52603,61300,86.95,67,22807,45,1492,01-06-1968,Salaried,16-09-2018,6,1998,1,0,0,1,0,0,818,A-Very Low Risk,1,0,0,0,0,0,0,0,0,0,0,0,2608,0,0,0,1yrs 7mon,1yrs 7mon,0,0


In [43]:
#Convert Account Age and Credit History Length columns to months

default_data['AVG_ACCT_AGE_MOS']= (default_data['AVERAGE_ACCT_AGE'].str.split(pat='yrs',n=1).str[0].astype('int')*12)+(default_data['AVERAGE_ACCT_AGE'].str.split(pat='yrs',n=1).str[1].str.split(pat='mon').str[0].astype('int'))

default_data['CREDIT_HIST_MOS']= (default_data['CREDIT_HISTORY_LENGTH'].str.split(pat='yrs',n=1).str[0].astype('int')*12)+(default_data['CREDIT_HISTORY_LENGTH'].str.split(pat='yrs',n=1).str[1].str.split(pat='mon').str[0].astype('int'))

default_data.drop(columns=['CREDIT_HISTORY_LENGTH', 'AVERAGE_ACCT_AGE'], inplace=True)

In [44]:
#Calculate (DOB - Disbursal Date) and convert to Days

born = pd.to_datetime(default_data['DATE_OF_BIRTH'],format = '%d-%m-%Y')
disbursed = pd.to_datetime(default_data['DISBURSAL_DATE'],format = '%d-%m-%Y')

default_data['AGE_IN_DAY_AT_DISBURSE'] = (disbursed - born).dt.days

default_data.drop(columns=['DATE_OF_BIRTH','DISBURSAL_DATE'], inplace=True)


In [45]:
#Drop unique identifiers

default_data.drop(columns=['UNIQUEID','CURRENT_PINCODE_ID','EMPLOYEE_CODE_ID'], inplace=True)

In [46]:
default_data.head()

,DISBURSED_AMOUNT,ASSET_COST,LTV,BRANCH_ID,SUPPLIER_ID,MANUFACTURER_ID,EMPLOYMENT_TYPE,STATE_ID,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,VOTERID_FLAG,DRIVING_FLAG,PASSPORT_FLAG,PERFORM_CNS_SCORE,PERFORM_CNS_SCORE_DESCRIPTION,PRI_NO_OF_ACCTS,PRI_ACTIVE_ACCTS,PRI_OVERDUE_ACCTS,PRI_CURRENT_BALANCE,PRI_SANCTIONED_AMOUNT,PRI_DISBURSED_AMOUNT,SEC_NO_OF_ACCTS,SEC_ACTIVE_ACCTS,SEC_OVERDUE_ACCTS,SEC_CURRENT_BALANCE,SEC_SANCTIONED_AMOUNT,SEC_DISBURSED_AMOUNT,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,NEW_ACCTS_IN_LAST_SIX_MONTHS,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,NO_OF_INQUIRIES,LOAN_DEFAULT,AVG_ACCT_AGE_MOS,CREDIT_HIST_MOS,AGE_IN_DAY_AT_DISBURSE
0,50578,58400,89.55,67,22807,45,Salaried,6,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12633
1,47145,65550,73.23,67,22807,45,Self employed,6,1,1,0,0,0,0,598,I-Medium Risk,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,0,1,23,23,12110
2,53278,61360,89.63,67,22807,45,Self employed,6,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12030
3,57513,66113,88.48,67,22807,45,Self employed,6,1,1,0,0,0,0,305,L-Very High Risk,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,1,1,8,15,9066
4,52378,60300,88.39,67,22807,45,Self employed,6,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,14901


In [47]:
# Time for encoding

# First some features are numeric that should be treated as categorical...

default_data = default_data.astype({'BRANCH_ID': 'object', 'SUPPLIER_ID': 'object','MANUFACTURER_ID': 'object', 'STATE_ID' : 'object'})

In [49]:
# Ready to run in the models!

default_model_data = pd.get_dummies(default_data)

default_model_data.head()

,DISBURSED_AMOUNT,ASSET_COST,LTV,MOBILENO_AVL_FLAG,AADHAR_FLAG,PAN_FLAG,VOTERID_FLAG,DRIVING_FLAG,PASSPORT_FLAG,PERFORM_CNS_SCORE,PRI_NO_OF_ACCTS,PRI_ACTIVE_ACCTS,PRI_OVERDUE_ACCTS,PRI_CURRENT_BALANCE,PRI_SANCTIONED_AMOUNT,PRI_DISBURSED_AMOUNT,SEC_NO_OF_ACCTS,SEC_ACTIVE_ACCTS,SEC_OVERDUE_ACCTS,SEC_CURRENT_BALANCE,SEC_SANCTIONED_AMOUNT,SEC_DISBURSED_AMOUNT,PRIMARY_INSTAL_AMT,SEC_INSTAL_AMT,NEW_ACCTS_IN_LAST_SIX_MONTHS,DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS,NO_OF_INQUIRIES,LOAN_DEFAULT,AVG_ACCT_AGE_MOS,CREDIT_HIST_MOS,AGE_IN_DAY_AT_DISBURSE,BRANCH_ID_1,BRANCH_ID_2,BRANCH_ID_3,BRANCH_ID_5,BRANCH_ID_7,BRANCH_ID_8,BRANCH_ID_9,BRANCH_ID_10,BRANCH_ID_11,BRANCH_ID_13,BRANCH_ID_14,BRANCH_ID_15,BRANCH_ID_16,BRANCH_ID_17,BRANCH_ID_18,BRANCH_ID_19,BRANCH_ID_20,BRANCH_ID_29,BRANCH_ID_34,BRANCH_ID_35,BRANCH_ID_36,BRANCH_ID_42,BRANCH_ID_43,BRANCH_ID_48,BRANCH_ID_61,BRANCH_ID_62,BRANCH_ID_63,BRANCH_ID_64,BRANCH_ID_65,BRANCH_ID_66,BRANCH_ID_67,BRANCH_ID_68,BRANCH_ID_69,BRANCH_ID_70,BRANCH_ID_72,BRANCH_ID_73,BRANCH_ID_74,BRANCH_ID_76,BRANCH_ID_77,BRANCH_ID_78,BRANCH_ID_79,BRANCH_ID_82,BRANCH_ID_84,BRANCH_ID_85,BRANCH_ID_97,BRANCH_ID_100,BRANCH_ID_101,BRANCH_ID_103,BRANCH_ID_104,BRANCH_ID_105,BRANCH_ID_111,BRANCH_ID_117,BRANCH_ID_120,BRANCH_ID_121,BRANCH_ID_130,BRANCH_ID_135,BRANCH_ID_136,BRANCH_ID_138,BRANCH_ID_142,BRANCH_ID_146,BRANCH_ID_147,BRANCH_ID_152,BRANCH_ID_153,BRANCH_ID_158,BRANCH_ID_159,BRANCH_ID_160,BRANCH_ID_162,BRANCH_ID_165,BRANCH_ID_202,BRANCH_ID_207,BRANCH_ID_217,BRANCH_ID_248,BRANCH_ID_249,BRANCH_ID_250,BRANCH_ID_251,BRANCH_ID_254,BRANCH_ID_255,BRANCH_ID_257,BRANCH_ID_258,BRANCH_ID_259,BRANCH_ID_260,BRANCH_ID_261,SUPPLIER_ID_10524,SUPPLIER_ID_12311,SUPPLIER_ID_12312,SUPPLIER_ID_12374,SUPPLIER_ID_12441,SUPPLIER_ID_12456,SUPPLIER_ID_12500,SUPPLIER_ID_12534,SUPPLIER_ID_12539,SUPPLIER_ID_12797,SUPPLIER_ID_12842,SUPPLIER_ID_12878,SUPPLIER_ID_13131,SUPPLIER_ID_13295,SUPPLIER_ID_13317,SUPPLIER_ID_13448,SUPPLIER_ID_13512,SUPPLIER_ID_13612,SUPPLIER_ID_13890,SUPPLIER_ID_13895,SUPPLIER_ID_13909,SUPPLIER_ID_13913,SUPPLIER_ID_13924,SUPPLIER_ID_13926,SUPPLIER_ID_13928,SUPPLIER_ID_13929,SUPPLIER_ID_13931,SUPPLIER_ID_13937,SUPPLIER_ID_13941,SUPPLIER_ID_13942,SUPPLIER_ID_13947,SUPPLIER_ID_13948,SUPPLIER_ID_13960,SUPPLIER_ID_13968,SUPPLIER_ID_13971,SUPPLIER_ID_13972,SUPPLIER_ID_13975,SUPPLIER_ID_13978,SUPPLIER_ID_13984,SUPPLIER_ID_13988,SUPPLIER_ID_13990,SUPPLIER_ID_13992,SUPPLIER_ID_13993,SUPPLIER_ID_14002,SUPPLIER_ID_14004,SUPPLIER_ID_14005,SUPPLIER_ID_14011,SUPPLIER_ID_14015,SUPPLIER_ID_14021,SUPPLIER_ID_14028,SUPPLIER_ID_14046,SUPPLIER_ID_14070,SUPPLIER_ID_14074,SUPPLIER_ID_14078,SUPPLIER_ID_14081,SUPPLIER_ID_14088,SUPPLIER_ID_14094,SUPPLIER_ID_14095,SUPPLIER_ID_14100,SUPPLIER_ID_14108,SUPPLIER_ID_14112,SUPPLIER_ID_14114,SUPPLIER_ID_14115,SUPPLIER_ID_14123,SUPPLIER_ID_14124,SUPPLIER_ID_14132,SUPPLIER_ID_14133,SUPPLIER_ID_14134,SUPPLIER_ID_14136,SUPPLIER_ID_14142,SUPPLIER_ID_14143,SUPPLIER_ID_14144,SUPPLIER_ID_14145,SUPPLIER_ID_14147,SUPPLIER_ID_14148,SUPPLIER_ID_14151,SUPPLIER_ID_14152,SUPPLIER_ID_14158,SUPPLIER_ID_14159,SUPPLIER_ID_14166,SUPPLIER_ID_14176,SUPPLIER_ID_14178,SUPPLIER_ID_14180,SUPPLIER_ID_14181,SUPPLIER_ID_14188,SUPPLIER_ID_14189,SUPPLIER_ID_14198,SUPPLIER_ID_14203,SUPPLIER_ID_14213,SUPPLIER_ID_14214,SUPPLIER_ID_14231,SUPPLIER_ID_14234,SUPPLIER_ID_14237,SUPPLIER_ID_14241,SUPPLIER_ID_14255,SUPPLIER_ID_14271,SUPPLIER_ID_14292,SUPPLIER_ID_14293,SUPPLIER_ID_14300,SUPPLIER_ID_14305,SUPPLIER_ID_14313,SUPPLIER_ID_14320,SUPPLIER_ID_14331,SUPPLIER_ID_14336,SUPPLIER_ID_14343,SUPPLIER_ID_14347,SUPPLIER_ID_14356,SUPPLIER_ID_14360,SUPPLIER_ID_14364,SUPPLIER_ID_14368,SUPPLIER_ID_14370,SUPPLIER_ID_14375,SUPPLIER_ID_14383,SUPPLIER_ID_14387,SUPPLIER_ID_14419,SUPPLIER_ID_14427,SUPPLIER_ID_14430,SUPPLIER_ID_14432,SUPPLIER_ID_14436,SUPPLIER_ID_14441,SUPPLIER_ID_14443,SUPPLIER_ID_14444,SUPPLIER_ID_14458,SUPPLIER_ID_14462,SUPPLIER_ID_14464,SUPPLIER_ID_14470,SUPPLIER_ID_14518,SUPPLIER_ID_14522,SUPPLIER_ID_14539,SUPPLIER_ID_14541,SUPPLIER_ID_14545,SUPP

In [50]:
len(default_model_data.columns)

3121

In [51]:
default_model_data.columns

Index(['DISBURSED_AMOUNT', 'ASSET_COST', 'LTV', 'MOBILENO_AVL_FLAG',
       'AADHAR_FLAG', 'PAN_FLAG', 'VOTERID_FLAG', 'DRIVING_FLAG',
       'PASSPORT_FLAG', 'PERFORM_CNS_SCORE',
       ...
       'PERFORM_CNS_SCORE_DESCRIPTION_K-High Risk',
       'PERFORM_CNS_SCORE_DESCRIPTION_L-Very High Risk',
       'PERFORM_CNS_SCORE_DESCRIPTION_M-Very High Risk',
       'PERFORM_CNS_SCORE_DESCRIPTION_No Bureau History Available',
       'PERFORM_CNS_SCORE_DESCRIPTION_Not Scored: More than 50 active Accounts found',
       'PERFORM_CNS_SCORE_DESCRIPTION_Not Scored: No Activity seen on the customer (Inactive)',
       'PERFORM_CNS_SCORE_DESCRIPTION_Not Scored: No Updates available in last 36 months',
       'PERFORM_CNS_SCORE_DESCRIPTION_Not Scored: Not Enough Info available on the customer',
       'PERFORM_CNS_SCORE_DESCRIPTION_Not Scored: Only a Guarantor',
       'PERFORM_CNS_SCORE_DESCRIPTION_Not Scored: Sufficient History Not Available'],
      dtype='object', length=3121)

# Model Testing

## GOSDT

In [56]:
from sklearn.preprocessing import OneHotEncoder
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/monks-problems/monks-1.train", sep=" ", header=None)
df = df.iloc[:,1:-1]

# assign column names 
df.columns = ["label", "head_shape", "body_shape", "is_smiling", "holding", "jacket_color", "has_tie"]
X = df.iloc[:,1:]
y = df.iloc[:,0]

# Encode the categorical features to binary features
enc = OneHotEncoder()
enc.fit(X)
X_new = pd.DataFrame(enc.transform(X).toarray(), dtype=int)
X_new.columns = ["head_shape_round", "head_shape_square", "head_shape_octagon", 
                 "body_shape_round", "body_shape_square", "body_shape_octagon",
                 "is_smiling_yes", "is_smiling_no", 
                 "holding_sword", "holding_balloon", "holding_flag",
                 "jacket_color_red", "jacket_color_yellow", "jacket_color_green", "jacket_color_blue",
                 "has_tie_yes", "has_tie_no"
                ]
X_new

,head_shape_round,head_shape_square,head_shape_octagon,body_shape_round,body_shape_square,body_shape_octagon,is_smiling_yes,is_smiling_no,holding_sword,holding_balloon,holding_flag,jacket_color_red,jacket_color_yellow,jacket_color_green,jacket_color_blue,has_tie_yes,has_tie_no
0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0
1,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1
2,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0
3,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1
4,1,0,0,1,0,0,0,1,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,0,1
120,0,0,1,0,0,1,0,1,0,0,1,1,0,0,0,0,1
121,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1
122,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1


In [57]:
# set GOSDT configuration
config = {
            "regularization": 0.02, # regularization penalizes the tree with more leaves. We recommend to set it to relative high value to find a sparse tree.  
            "depth_budget": 6,
            "time_limit": 60, # training time limit in seconds
            "similar_support": False
        }

model = GOSDT(config)

# train GOSDT 
model.fit(X_new, y)
print("evaluate the model, extracting tree and scores", flush=True)

acc = model.score(X_new, y)
n_leaves = model.leaves()
n_nodes = model.nodes()
time = model.utime

print("Model training time: {}".format(time))
print("Training accuracy: {}".format(acc))
print("# of leaves: {}".format(n_leaves))
print(model.tree)

gosdt reported successful execution
training completed. 0.000/0.000/0.574 (user, system, wall), mem=0 MB
bounds: [0.140000..0.140000] (0.000000) loss=0.000000, iterations=22336
evaluate the model, extracting tree and scores
Model training time: 0.0
Training accuracy: 1.0
# of leaves: 7
if jacket_color_red = 1 then:
    predicted class: 1
    misclassification penalty: 0.0
    complexity penalty: 0.02

else if body_shape_round = 1 and head_shape_round = 1 and jacket_color_red != 1 then:
    predicted class: 1
    misclassification penalty: 0.0
    complexity penalty: 0.02

else if body_shape_round != 1 and head_shape_round = 1 and jacket_color_red != 1 then:
    predicted class: 0
    misclassification penalty: 0.0
    complexity penalty: 0.02

else if body_shape_square = 1 and head_shape_round != 1 and head_shape_square = 1 and jacket_color_red != 1 then:
    predicted class: 1
    misclassification penalty: 0.0
    complexity penalty: 0.02

else if body_shape_square != 1 and head_shap

## OCT

In [58]:
X, y = np.arange(10).reshape((5, 2)), range(5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [59]:
octree = miptree.optimalDecisionTreeClassifier(max_depth=3, min_samples_split=2, alpha=0.01, warmstart=True, timelimit=600, output=True)
octree.fit(X_train, y_train)
y_test_pred = octree.predict(X_test)

Training data include 3 instances, 2 features.
Set parameter TimeLimit to value 600
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 215 rows, 124 columns and 767 nonzeros
Model fingerprint: 0x49b82c63
Variable types: 47 continuous, 77 integer (77 binary)
Coefficient statistics:
  Matrix range     [1e-01, 3e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint R85 by 1.000000000

Found heuristic solution: objective 2.0000000
Presolve removed 215 rows and 124 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000

## Testing GOSDT on Dataset

In [60]:
y = default_model_data.LOAN_DEFAULT
x = default_model_data.drop(columns=['LOAN_DEFAULT'])

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.99985, random_state=42)

len(X_train)

34

In [64]:
%%timeit -n 1 -r 1

config = {
            "regularization": 0.05, # regularization penalizes the tree with more leaves. We recommend to set it to relative high value to find a sparse tree.  
            "depth_budget": 5,
            "time_limit": 0, # training time limit in seconds
            "similar_support": True,
            "feature_exchange": True
        }

model = GOSDT(config)

# train GOSDT 
model = model.fit(X_train, y_train)
print("evaluate the model, extracting tree and scores", flush=True)

acc = model.score(X_train, y_train)
n_leaves = model.leaves()
n_nodes = model.nodes()
time = model.utime

print("Model training time: {}".format(time))
print("Training accuracy: {}".format(acc))
print("# of leaves: {}".format(n_leaves))
print(model.tree)

gosdt reported successful execution
training completed. 0.000/0.000/0.532 (user, system, wall), mem=0 MB
bounds: [0.217647..0.217647] (0.000000) loss=0.117647, iterations=722
evaluate the model, extracting tree and scores
Model training time: 0.0
Training accuracy: 0.8823529411764706
# of leaves: 2
if 89.684998 <= LTV then:
    predicted class: 1
    misclassification penalty: 0.0
    complexity penalty: 0.05

else if LTV < 89.684998 then:
    predicted class: 0
    misclassification penalty: 0.118
    complexity penalty: 0.05
635 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit -n 1 -r 1

# I tried playing around with other hyperparameters like worker_limit, upperbound, and feature_transform
# (as we had to do the encoding manually), but it didn't seem to make a difference. The main culprit seems
# to be the dataset size.

config = {
            "regularization": 0.05,
            "depth_budget": 5,
            "reference_LB": False, 
            "path_to_labels": "",
            "time_limit": 0,
            "uncertainty_tolerance": 0.0,
            "upperbound": 0,
            "worker_limit": 1,
            "stack_limit": 0,
            "precision_limit": 0,
            "model_limit": 1,
            "verbose": False,
            "diagnostics": False,
            "balance": False,
            "look_ahead": True,
            "similar_support": True,
            "cancellation": True,
            "continuous_feature_exchange": False,
            "feature_exchange": False,
            "feature_transform": True,
            "rule_list": False,
            "non_binary": False,
            "costs": "",
            "model": "",
            "timing": "",
            "trace": "",
            "tree": "",
            "profile": "",
        }

model = GOSDT(config)

# train GOSDT 
model = model.fit(X_train, y_train)
print("evaluate the model, extracting tree and scores", flush=True)

acc = model.score(X_train, y_train)
n_leaves = model.leaves()
n_nodes = model.nodes()
time = model.utime

print("Model training time: {}".format(time))
print("Training accuracy: {}".format(acc))
print("# of leaves: {}".format(n_leaves))
print(model.tree)